# Drifting y Quality

Un desafío común para los **data scientists** en el mundo real es que, a diferencia de las competencia de **Kaggle**, los datos cambian con el tiempo. Esto significa que los datos que recibiremos en el futuro no necesariamente reflejarán los que usamos para entrenar nuestros modelos. Esta homogeneidad es esencial, ya que los algoritmos aprenden patrones en función de las distribuciones observadas durante el entrenamiento. Si esas distribuciones cambian, la capacidad predictiva del modelo se deteriora.

Podemos identificar dos tipos principales de problemáticas asociadas a estos cambios:

- **Data Quality (DQ)**
- **Data Drift (DF)**

Comencemos con el primero: **Data Quality (DQ)**. Hay mucho que decir sobre este tema, pero mencionaremos brevemente que los datos provienen de procesos creados por seres humanos, quienes son propensos a cometer errores y hacer modificaciones. A lo largo del ciclo de vida de los modelos, es común que se enfrenten a este tipo de situaciones. Por ello, es fundamental implementar controles que garanticen que los datos a los que se aplican los modelos mantienen la misma estructura que los datos usados en el entrenamiento.

A continuación, veremos dos indicadores clave para monitorear la calidad de los datos, aunque no son los únicos a considerar.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

from os import path
# pd.set_option('display.max_rows', None)

# # Mostrar todas las columnas
# pd.set_option('display.max_columns', None)


c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'
dataset_path = base_path + 'datasets/'
dataset_file = 'competencia_01_shap.csv'

# agregue sus semillas
semillas = [540079, 250829, 314299, 302111, 801007]

data = pd.read_csv(dataset_path + dataset_file)

In [3]:
df=data.copy()

In [4]:
df=df.sort_values(by=['foto_mes', 'numero_de_cliente'])

## NaN por foto_mes

In [5]:
def calcular_porcentaje_nan_por_mes(dataset):
    # Crear una lista de las variables que quiero analizar los NaN
    variables = dataset.columns.difference(['numero_de_cliente', 'foto_mes', 'clase_ternaria'])

    # Inicializar una lista para almacenar los resultados
    resultados = []

    # Agrupar el dataset por 'foto_mes' y calcular el ratio de NaN para cada columna
    for mes, datos_mes in dataset.groupby('foto_mes'):
        # Calcular el porcentaje de NaN para cada columna en este mes
        porcentaje_nan = datos_mes[variables].isna().mean() * 100
        # Crear un DataFrame temporal para almacenar los resultados
        resultado_mes = pd.DataFrame({
            'foto_mes': mes,
            'campo': porcentaje_nan.index,
            'porcentaje_nan': porcentaje_nan.values
        })
        # Añadir los resultados del mes a la lista
        resultados.append(resultado_mes)

    # Concatenar todos los resultados en un único DataFrame
    tabla_resultados_nan = pd.concat(resultados, ignore_index=True)

    return tabla_resultados_nan

# Llamada a la función
tabla_porcentajes_nan_por_mes = calcular_porcentaje_nan_por_mes(df)

In [6]:
tabla_pivoteada_nan = tabla_porcentajes_nan_por_mes.pivot(index='campo', columns='foto_mes', values='porcentaje_nan')
tabla_pivoteada_nan['total_nan'] = tabla_pivoteada_nan.sum(axis=1)/6

# Ordenar la tabla por la columna 'total_nan' en orden descendente
tabla_ordenada_nan = tabla_pivoteada_nan.sort_values(by='total_nan', ascending=False)
tabla_ordenada_nan.head(20)

foto_mes,202101,202102,202103,202104,202105,202106,total_nan
campo,,,,,,,
Master_cconsumos,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_mconsumototal,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_mconsumospesos,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_mpagosdolares,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Master_mpagospesos,60.885290,60.002705,59.844824,59.780608,59.299733,58.459691,59.712142
Visa_mpagosdolares,14.053917,13.745189,13.733085,13.576086,13.449518,13.081952,13.606625
Visa_mpagospesos,14.053917,13.745189,13.733085,13.576086,13.449518,13.081952,13.606625
Visa_cconsumos,14.053917,13.745189,13.733085,13.576086,13.449518,13.081952,13.606625
Visa_mconsumototal,14.053917,13.745189,13.733085,13.576086,13.449518,13.081952,13.606625


In [8]:
df

,mpayroll,cpayroll_trx,mprestamos_personales,mcuentas_saldo,mrentabilidad_annual,mactivos_margen,mtarjeta_visa_consumo,ctrx_quarter,mpasivos_margen,mrentabilidad,...,Master_mpagosdolares,ctarjeta_visa_descuentos,cprestamos_prendarios,ctarjeta_visa,Master_msaldodolares,mttarjeta_master_debitos_automaticos,Master_mconsumototal,tmobile_app,foto_mes,clase_ternaria
366138,140760.00,2,0.0,71493.86,17994.40,-312.22,119804.35,166,362.63,1664.94,...,0.00,0,0,1,0.00,8026.47,30487.50,0,202101,CONTINUA
793371,0.00,0,0.0,74024.15,18640.95,748.78,41362.91,145,2431.70,4879.78,...,2.34,0,0,1,6414.32,0.00,31393.77,0,202101,CONTINUA
716051,115229.65,1,0.0,-1467.41,37789.01,310.11,9838.70,182,728.45,1828.35,...,NaN,0,0,1,NaN,0.00,NaN,0,202101,CONTINUA
919005,0.00,0,0.0,5.05,2600.67,-181.19,7517.73,25,95.64,175.87,...,NaN,0,0,1,0.00,0.00,NaN,0,202101,CONTINUA
746245,0.00,0,0.0,31576.35,32270.07,-902.69,41431.89,32,1101.80,16600.69,...,NaN,0,0,1,0.00,0.00,NaN,0,202101,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832252,23460.00,1,0.0,0.00,0.00,0.00,0.00,2,0.00,0.00,...,NaN,0,0,1,0.00,0.00,NaN,0,202106,NaN
471992,0.00,1,0.0,91576.57,95.20,0.00,0.00,5,81.16,95.20,...,NaN,0,0,1,0.00,0.00,NaN,0,202106,NaN
259351,0.00,1,0.0,71850.94,72.08,0.00,0.00,2,61.45,72.08,...,NaN,0,0,1,0.00,0.00,NaN,0,202106,NaN
903648,116570.62,2,0.0,0.00,0.00,0.00,0.00,2,0.00,0.00,...,NaN,0,0,1,0.00,0.00,NaN,0,202106,NaN


In [11]:
col_eliminar_nan=["Visa_fultimo_cierre","Master_fultimo_cierre", "cprestamos_personales","mprestamos_personales"]

In [12]:
df.drop(columns=col_eliminar_nan, axis=1,inplace=True)

In [78]:
def calcular_porcentaje_ceros_por_mes(dataset):
    # Crear una lista de las variables que quiero analizar los ceros
    variables = dataset.columns.difference(['numero_de_cliente', 'foto_mes', 'clase_ternaria'])

    # Inicializar una lista para almacenar los resultados
    resultados = []

    # Agrupar el dataset por 'foto_mes' y calcular el ratio de ceros para cada columna
    for mes, datos_mes in dataset.groupby('foto_mes'):
        # Calcular el porcentaje de ceros para cada columna en este mes
        porcentaje_ceros = (datos_mes[variables] == 0).mean() * 100
        # Crear un DataFrame temporal para almacenar los resultados
        resultado_mes = pd.DataFrame({
            'foto_mes': mes,
            'campo': porcentaje_ceros.index,
            'porcentaje_ceros': porcentaje_ceros.values
        })
        # Añadir los resultados del mes a la lista
        resultados.append(resultado_mes)

    # Concatenar todos los resultados en un único DataFrame
    tabla_resultados = pd.concat(resultados, ignore_index=True)

    return tabla_resultados

# Llamada a la función
tabla_porcentajes_ceros_por_mes = calcular_porcentaje_ceros_por_mes(df)


In [79]:
tabla_pivoteada = tabla_porcentajes_ceros_por_mes.pivot(index='campo', columns='foto_mes', values='porcentaje_ceros')
tabla_pivoteada['total_zero'] = tabla_pivoteada.sum(axis=1)/6

# Ordenar la tabla por la columna 'total_zero' en orden descendente
tabla_pivoteada_zero = tabla_pivoteada.sort_values(by='total_zero', ascending=False)
tabla_pivoteada_zero.head(40)

foto_mes,202101,202102,202103,202104,202105,202106,total_zero
campo,,,,,,,
mcuenta_corriente_adicional,99.937047,99.937902,99.937074,99.937839,99.939255,99.940561,99.938280
cpayroll2_trx,99.875946,99.889330,99.904084,99.909806,99.907668,99.871419,99.893042
mpayroll2,99.875946,99.889330,99.904084,99.909806,99.907668,99.871419,99.893042
ccheques_emitidos_rechazados,99.777196,99.822313,99.829551,99.834847,99.831737,99.833815,99.821576
mcheques_emitidos_rechazados,99.777196,99.822313,99.829551,99.834847,99.831737,99.833815,99.821576
mcheques_depositados_rechazados,99.781517,99.835225,99.829551,99.803157,99.811691,99.793178,99.809053
ccheques_depositados_rechazados,99.781517,99.835225,99.829551,99.803157,99.811691,99.793178,99.809053
cliente_vip,99.678447,99.728859,99.729358,99.728198,99.727863,99.434727,99.671242
minversion1_dolares,99.620431,99.626797,99.631609,99.636175,99.640390,99.646401,99.633634


In [65]:
# variables_zero= tabla_pivoteada_zero.head(40).index.tolist()

In [80]:
# col_eliminar_zero= ['mcuenta_corriente_adicional',
#  'mpayroll2',
#  'mcheques_emitidos_rechazados',
#  'mcheques_depositados_rechazados',
#  'minversion1_dolares',
#  'mforex_buy',
#  'mplazo_fijo_pesos',
#  'mpagodeservicios',
#  'mprestamos_prendarios',
#  'mprestamos_hipotecarios',
#  'mcaja_ahorro_adicional',
#  'mcheques_emitidos',
#  'mcajeros_propios_descuentos',
#  'minversion2',
#  'minversion1_pesos',
#  'mcheques_depositados',
#  'mtarjeta_master_descuentos',
#  'mforex_sell']

In [81]:
# df.drop(columns=col_eliminar_zero, axis=1,inplace=True)

In [13]:
df.shape

(981946, 98)

In [14]:
ruta_datasets = "../../../datasets/"
nombre_exp="competencia_01_DQ.csv"
df.to_csv(ruta_datasets+nombre_exp, index=False)